In [1]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser
import os


def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [2]:
headers = {
    'User-Agent': 'GOLUB CAPITAL BDC, Inc.'
}
filing_links = pd.read_excel(
    "../GBDC__sec_filing_links.xlsx")
filing_links.head(5)
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head(5)

,Form type,Form description,Filing date,Reporting date,Act,Film number,Accession number,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],"May 03, 2013","March 31, 2013",34,13810352,0001144204-13-026113,https://www.sec.gov/Archives/edgar/data/147676...
1,10-Q,Quarterly report [Sections 13 or 15(d)],"August 08, 2013","June 30, 2013",34,131020072,0001144204-13-043799,https://www.sec.gov/Archives/edgar/data/147676...
2,10-K,"Annual report [Section 13 and 15(d), not S-K I...","December 03, 2013","September 30, 2013",34,131254591,0001144204-13-065322,https://www.sec.gov/Archives/edgar/data/000147...
3,10-Q,Quarterly report [Sections 13 or 15(d)],"February 06, 2014","December 31, 2013",34,14578102,0001144204-14-006255,https://www.sec.gov/Archives/edgar/data/000147...
4,10-Q,Quarterly report [Sections 13 or 15(d)],"May 08, 2014","March 31, 2014",34,14823110,0001144204-14-028416,https://www.sec.gov/Archives/edgar/data/000147...


In [ ]:
# this one is the OG one

def extract_tables(soup_content, qtr_date):
    master_table = None
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern1, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern1, tag.next.text)
        print(date_str)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)

    return master_table


path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    master_table = extract_tables(content, qtr_date)
    master_table.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    writer.book.save(path)
writer.close()

In [3]:
filing_links = filing_links[filing_links['Reporting date']
                            != 'September 30, 2017']

In [ ]:
data_frames = []
data_frames_shapes = []


def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)
    if qtr_date == 'December 31, 2015' or qtr_date == 'June 30, 2016':
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*(\(.*\)|)\s*-.*\s*\(.*\)$')
    else:
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                # new_table = new_table.replace(
                #     r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                # new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
            # print(master_table)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table


# This does the job
path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/Test_GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    master_table = extract_tables(content, qtr_date)
    master_table.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    writer.book .save(path)
writer.close()

In [ ]:
data_frames = []
data_frames_shapes = []


def extract_tables(soup_content, qtr_date):
    master_table = None
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern1, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern1, tag.next.text)
        print(date_str)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                table = tag.find_next("table")
                if table:
                    # Extract the table data into a data frame
                    table_data = []
                    for row in table.find_all('tr'):
                        # Include header cells ('th') if necessary
                        columns = row.find_all(['th', 'td'])
                        row_data = [column.get_text(strip=True)
                                    for column in columns]
                        table_data.append(row_data)

                    # Create a data frame from the table data and add it to the list
                    table_df = pd.DataFrame(table_data)
                    new_table = table_df.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '0').replace('%', '')) if type(x) == str else x)

                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    table_df = new_table.dropna(how='all', axis=0)

                    if len(table_df.columns) > 10:
                        data_frames.append(table_df)

                        if master_table is None:
                            master_table = table_df
                        else:
                            master_table = pd.concat(
                                [master_table, table_df], ignore_index=True)

            # print(master_table)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table

In [ ]:
data_frames = []
data_frames_shapes = []


def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)
    if qtr_date == 'December 31, 2015' or qtr_date == 'June 30, 2016':
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*(\(.*\)|)\s*-.*\s*\(.*\)$')
    else:
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '0').replace('%', '')) if type(x) == str else x)

                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
            # print(master_table)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table

In [ ]:
def process_table_fun(soi_table_df, process_tables_shapes):
    print(1, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.replace(r'\n', '', regex=True)
    print(2, 'shape:', soi_table_df.shape)
    print(6, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace('—', 0)
    soi_table_df = soi_table_df.replace('-', 0)
    print(7, 'shape:', soi_table_df.shape)
    soi_table_df.reset_index(drop=True, inplace=True)

    soi_table_df = soi_table_df.applymap(
        lambda x: x.strip() if isinstance(x, str) else x)

    pattern = r'Total\s+Investments'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row
    if soi_table_df[matching_rows].index[0] < 20:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
            drop=True)
    else:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
            drop=True)

# removest end extra
    pattern = r'Net asset value per common share|How We Addressed the Matter in Our Audit'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)

    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax(
        )+1:].reset_index(drop=True)

    # removing all col name
    pattern = r'(?:Spread\s*Above|cost|Percentage|Above)'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    soi_table_df = soi_table_df[~matching_rows]

    print(0, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=1).reset_index(
        drop=True)
    print(3, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=0).reset_index(
        drop=True)
    print(4, 'shape:', soi_table_df.shape)
    soi_table_df.dropna().reset_index(
        drop=True)
    print(5, 'shape:', soi_table_df.shape)


# drops the sub total
    soi_table_df = soi_table_df.dropna(subset=[soi_table_df.columns[0]])

    num_cols = soi_table_df.shape[1]
    data_col_mapper = dict(zip(soi_table_df.columns.to_list(), [
        i for i in range(0, num_cols)]))
    soi_table_df = soi_table_df.rename(columns=data_col_mapper)

    for index, row in soi_table_df.iterrows():
        for column in soi_table_df.columns:
            if pd.isna(row[column]):
                # Find the next column in the same row
                next_column = soi_table_df.columns.get_loc(column) + 1
                if next_column < len(soi_table_df.columns):
                    next_column_name = soi_table_df.columns[next_column]
                    # Replace NaN value with the value from the next column
                    soi_table_df.at[index, column] = row[next_column_name]
                    # Set the next column to NaN
                    soi_table_df.at[index, next_column_name] = np.nan

    for index, row in soi_table_df.iterrows():
        for column in soi_table_df.columns:
            if pd.isna(row[column]):
                while True:
                    # Find the next column in the same row
                    next_column = soi_table_df.columns.get_loc(column) + 1
                    if next_column < len(soi_table_df.columns):
                        next_column_name = soi_table_df.columns[next_column]
                        # Replace NaN value with the value from the next column
                        soi_table_df.at[index, column] = row[next_column_name]
                        # Set the next column to NaN
                        soi_table_df.at[index, next_column_name] = np.nan
                        column = next_column_name
                    else:
                        # No more columns to replace, break out of the loop
                        break

    soi_table_df = soi_table_df.dropna(axis=1, thresh=10)
    soi_table_df = soi_table_df.dropna(how='all', axis=1).reset_index(
        drop=True)

    process_tables_shapes.append(soi_table_df.shape)
    print(soi_table_df.info())

    return soi_table_df


process_tables = {}
process_tables_shape = []

In [ ]:
# This does the job
path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/Tr_GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    master_table = extract_tables(content, qtr_date)
    process_table = process_table_fun(master_table, process_tables_shape)
    process_table.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    writer.book .save(path)
writer.close()


# last work Nov 6th 2023

In [ ]:
process_tables_shape

In [9]:
data_frames = []
data_frames_shapes = []


def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)
    if qtr_date == 'December 31, 2015' or qtr_date == 'June 30, 2016':
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*(\(.*\)|)\s*-.*\s*\(.*\)$')
    else:
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('-', '0').replace('%', '')) if type(x) == str else x)

                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
            # print(master_table)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table


# date = "December 31, 2013"
# url = filing_links[filing_links['Reporting date']
#                    == date]['Filings URL'].values[0]
# # webbrowser.open(url=url)
# response = requests.get(url=url, headers=headers)
# content = parse_and_trim(response.content, "HTML")
# master_table = extract_tables(content, date)
# contentDUP = content
# master_tableDUP = master_table


# This does the job
# path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/Master_tables_GBDC_Investment.xlsx'
# if not os.path.exists('../MT_csv_file'):
#     os.makedirs('../MT_csv_file')

# writer = pd.ExcelWriter(path, engine='openpyxl')
# for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
#     print(html_link, qtr_date)
#     response = requests.get(html_link, headers=headers)
#     content = parse_and_trim(response.content, 'HTML')
#     master_table = extract_tables(content, qtr_date)
#     master_table.to_excel(
#         writer, sheet_name=qtr_date.replace(',', ''), index=False)
#     master_table.to_csv(
#         '../MT_csv_file/'+qtr_date.replace(',', '')+'.csv')
#     writer.book .save(path)
# writer.close()

In [10]:
html_link = "https://www.sec.gov/Archives/edgar/data/1476765/000114420416079295/v430459_10q.htm"
qtr_date = "December 31 2015"
response = requests.get(html_link, headers=headers)
content = parse_and_trim(response.content, 'HTML')
master_table = extract_tables(content, qtr_date)

December 31 2015
December 31, 2015 december312015 december312015
September 30, 2015 december312015 september302015
(63, 27)


In [8]:
master_table.to_csv('test.csv')

In [11]:
data_frames = []
data_frames_shapes = []


def extract_tables(soup_content, qtr_date):
    master_table = None
    print(qtr_date)
    if qtr_date == 'December 31, 2015' or qtr_date == 'June 30, 2016':
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*(\(.*\)|)\s*-.*\s*\(.*\)$')
    else:
        consolidated_schedule_regex = re.compile(
            r'(?i)^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '0').replace('%', '')) if type(x) == str else x)

                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)
            # print(master_table)

            master_table = master_table.replace('N/A', 'No Value')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table


date = "December 31, 2015"
url = filing_links[filing_links['Reporting date']
                   == date]['Filings URL'].values[0]
# webbrowser.open(url=url)
response = requests.get(url=url, headers=headers)
content = parse_and_trim(response.content, "HTML")
master_table = extract_tables(content, date)
contentDUP = content
master_tableDUP = master_table
master_table.to_csv('test.csv')

filing_links = filing_links[filing_links['Reporting date']
                            != 'September 30, 2017']

# This does the job
# path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/Master_tables_GBDC_Investment.xlsx'
# if not os.path.exists('../MT_csv_file'):
#     os.makedirs('../MT_csv_file')

# writer = pd.ExcelWriter(path, engine='openpyxl')
# for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
#     print(html_link, qtr_date)
#     response = requests.get(html_link, headers=headers)
#     content = parse_and_trim(response.content, 'HTML')
#     master_table = extract_tables(content, qtr_date)
#     master_table.to_excel(
#         writer, sheet_name=qtr_date.replace(',', ''), index=False)
#     master_table.to_csv(
#         '../MT_csv_file/'+qtr_date.replace(',', '')+'.csv')
#     writer.book .save(path)
# writer.close()

December 31, 2015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
December 31, 2015 december312015 december312015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
September 30, 2015 december312015 september302015
(505, 27)
